In [13]:
import pandas as pd
import time
from datetime import datetime
import requests
from bs4 import BeautifulSoup
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [14]:
ticker_csv = pd.read_csv('ticker.csv')
tickers = ticker_csv['symbol'].tolist()

In [15]:
finviz_url = "https://finviz.com/quote.ashx?t="

In [16]:
#iterating through the tickers list and performing a get request to finviz for news titles related to that symbol.
#create a beautiful soup object for parsing.

ticker_news = {}
for ticker in tickers:
    url = f'{finviz_url}{ticker}'

    response = requests.get(url,headers={'user-agent':'sentiment_analysis'})
   
    soup = BeautifulSoup(response.text, 'html.parser')
    news_table = soup.find(id='news-table')
    ticker_news[ticker] = news_table

    #print(ticker_news)

In [17]:
#parsing through the ticker_news titles and storing the ticker, date, time and news title in a list.
parsed_data = []

for ticker, news_table in ticker_news.items():
    for row in news_table.find_all('tr'):
        
        title = row.a.text
        date_data = row.td.text.split(" ")
        date = date_data[0]
        
        parsed_data.append([ticker, date, title])

In [18]:
#creating a pandas df out of the parsed data list
sentiment_df = pd.DataFrame(parsed_data, columns=['ticker', 'date', 'news_title'])
sentiment_df['date'] = pd.to_datetime(sentiment_df['date']).dt.normalize()
sentiment_df.head()

,ticker,date,news_title
0,ACIW,2021-06-12,"Is ACI Worldwide, Inc. (NASDAQ:ACIW) Trading A..."
1,ACIW,2021-06-08,Indian Consumers More Concerned About Digital ...
2,ACIW,2021-06-01,ACI Worldwide Powers Payments Innovation for P...
3,ACIW,2021-05-28,ACI Worldwide to Attend Craig-Hallum Instituti...
4,ACIW,2021-05-25,ACI Worldwide and J.P. Morgan Collaborate to O...


In [19]:
#creating a sentiment analyzer object
vader = SentimentIntensityAnalyzer()

In [20]:
#lambda function to get the compound sentiment analysis score for each title.
sentiment = lambda title: vader.polarity_scores(title)['compound']

In [21]:
#adding an additional compound_score column to the ticker df for the compound sentiment analysis score for each title.
sentiment_df['compound_score'] = sentiment_df['news_title'].apply(sentiment).round(3)

In [22]:
sentiment_df.head()

,ticker,date,news_title,compound_score
0,ACIW,2021-06-12,"Is ACI Worldwide, Inc. (NASDAQ:ACIW) Trading A...",0.000
1,ACIW,2021-06-08,Indian Consumers More Concerned About Digital ...,-0.586
2,ACIW,2021-06-01,ACI Worldwide Powers Payments Innovation for P...,0.296
3,ACIW,2021-05-28,ACI Worldwide to Attend Craig-Hallum Instituti...,0.000
4,ACIW,2021-05-25,ACI Worldwide and J.P. Morgan Collaborate to O...,0.361


In [23]:
sentiment_df.to_csv('sentiment.csv')